In [3]:
import numpy as np
from copy import copy
import matplotlib.pyplot as plt

In [102]:
def generate_HMM_observation(num_obs, pi, T, E):
    def drawFrom(probs):
        return np.where(np.random.multinomial(1,probs) == 1)[0][0]

    obs = np.zeros(num_obs)
    states = np.zeros(num_obs)
    states[0] = drawFrom(pi)
    obs[0] = drawFrom(E[:, int(states[0])])
    for t in range(1,num_obs):
        states[t] = drawFrom(T[int(states[t-1]),:])
        obs[t] = drawFrom(E[:, int(states[t])])
    return obs, states

In [103]:
True_pi = np.array([0.3, 0.7])
True_T = np.array([[0.15, 0.85],
                  [0.88, 0.12]])

In [118]:
True_E = np.array([[0.6, 0.4],
                   [0.2, 0.3],
                   [0.2, 1.3]])

In [134]:
num_obs = 4
obs, states = generate_HMM_observation(num_obs, True_pi, True_T, True_E)
print obs, states

[ 0.  2.  2.  1.] [ 0.  1.  0.  1.]


In [135]:
print obs, states

[ 0.  2.  2.  1.] [ 0.  1.  0.  1.]


In [136]:
True_E = True_E.T

In [137]:
True_E

array([[ 0.6,  0.2,  0.2],
       [ 0.4,  0.3,  1.3]])

In [138]:
nStates = True_T.shape[0]
nSamples = len(obs)

A = True_T
B = True_E
pi = copy(True_pi)

In [172]:
# alpha_t(i) = P(O_1 O_2 ... O_t, q_t = S_i | hmm)
# Initialize alpha
alpha = np.zeros((nStates,nSamples))
c = np.zeros(nSamples) #scale factors
alpha[:,0] = pi.T * B[:,int(obs[0])]
c[0] = 1.0/np.sum(alpha[:,0])
alpha[:,0] = c[0] * alpha[:,0]
# Update alpha for each observation step
for t in range(1,nSamples):
    alpha[:,t] = np.dot(alpha[:,t-1].T, A).T * B[:,int(obs[t])]
    c[t] = 1.0/np.sum(alpha[:,t])
    alpha[:,t] = c[t] * alpha[:,t]

# Initialize beta
beta = np.zeros((nStates,nSamples))
beta[:,nSamples-1] = 1
beta[:,nSamples-1] = c[nSamples-1] * beta[:,nSamples-1]
# Update beta backwards from end of sequence
for t in range(len(obs)-1,0,-1):
    beta[:,t-1] = np.dot(A, (B[:,int(obs[t])] * beta[:,t]))
    beta[:,t-1] = c[t-1] * beta[:,t-1]

In [176]:
xi = np.zeros((nStates,nStates,nSamples-1));
for t in range(nSamples-1):
    denom = np.dot(np.dot(alpha[:,t].T, A) * B[:,int(obs[t+1])].T,
                   beta[:,t+1])
    for i in range(nStates):
        numer = alpha[i,t] * A[i,:] * B[:,int(obs[t+1])].T * beta[:,t+1].T
        xi[i,:,t] = numer / denom


In [178]:
xi.T

array([[[ 0.0391778 ,  0.3575337 ],
        [ 0.49465777,  0.10863073]],

       [[ 0.01396929,  0.36357352],
        [ 0.38274221,  0.23971498]],

       [[ 0.03974135,  0.51675692],
        [ 0.33780146,  0.10570028]]])

In [ ]:
0.51211628  0.34534506